In [9]:
def clamp(n, minimum, maximum):
    return max(min(maximum, n), minimum)

In [159]:
#This cell graphs CO2 emissions, population and year
import csv, pandas as pd, numpy as np
from collections import defaultdict

data = []

with open('data/1990-2018 GHG Emissions.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        data.append(line)
        
print(data)
graph = pd.DataFrame(data)
graph.ylim(0, 30)
graph.plot.scatter(x = 'Year', y = 'Total')

[{'Year': '1990', 'Buildings': '14.6', 'Transportation': '7.3', 'Waste': '4.1', 'Total': '26'}, {'Year': '2004', 'Buildings': '14.3', 'Transportation': '8.2', 'Waste': '1.4', 'Total': '23.9'}, {'Year': '2008', 'Buildings': '13.5', 'Transportation': '8.4', 'Waste': '1.4', 'Total': '23.3'}, {'Year': '2011', 'Buildings': '11.1', 'Transportation': '8.4', 'Waste': '1', 'Total': '20.5'}, {'Year': '2012', 'Buildings': '9.9', 'Transportation': '8.4', 'Waste': '0.9', 'Total': '19.2'}, {'Year': '2013', 'Buildings': '10.1', 'Transportation': '7', 'Waste': '0.9', 'Total': '18'}, {'Year': '2014', 'Buildings': '9.7', 'Transportation': '6.3', 'Waste': '1.7', 'Total': '17.7'}, {'Year': '2015', 'Buildings': '9', 'Transportation': '6.4', 'Waste': '1.9', 'Total': '17.3'}, {'Year': '2016', 'Buildings': '8.1', 'Transportation': '6.4', 'Waste': '1.6', 'Total': '16.1'}, {'Year': '2017', 'Buildings': '7.9', 'Transportation': '5.7', 'Waste': '1.5', 'Total': '15.1'}, {'Year': '2018', 'Buildings': '8.9', 'Transp

AttributeError: 'DataFrame' object has no attribute 'ylim'

In [3]:
#This cell organizes the data for renewable installations
import csv, re, math
from collections import defaultdict

renewable = {}
with open('data/Renewable Energy Installations.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

    for line in csv_reader:
        name = line['LINEAR_NAME_FULL']
        geo = line['geometry']
        
        renewable.setdefault(name, {
            'Latitude': 0,
            'Longitude': 0
        })
        
        #Gathers coordinates of renewables
        x = re.search(r"\(([A-Za-z0-9.\-,\s]+)\)", geo)
        coordinates = x.group(1).split(", ")
        
        renewable[name]['Latitude'] = coordinates[1]
        renewable[name]['Longitude'] = coordinates[0]

for i, value in renewable.items():
    #print(i)
    a = renewable[i]['Latitude']
    b = renewable[i]['Longitude']
    #print(f'\t{a}, {b}')

In [4]:
#This cell organizes the data for CO2 emissions
import csv, pgeocode, pandas as pd
from collections import defaultdict

building = {}

with open('data/Annual Energy.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        #print(line)
        name = line['\ufeffOperation Name']
        if not name == "":
            if (not line['Total (calculated in webform)'].replace(",","") == ""):
                emissions = float(line['Total (calculated in webform)'].replace(",",""))
            else:
                emissions = 0
            postal = line['Postal Code']
            nomi = pgeocode.Nominatim('ca')
            
            energy = float(line['Energy Type and Amount Purchased and Consumed in Natural Units'].replace(",", ""))
        
            building.setdefault(name, {
                'Emissions': 0,
                'Latitude': 0,
                'Longitude': 0,
                'Energy': 0,
                'Efficiency': 0
            })
        
            nomi = nomi.query_postal_code(postal)
            #Sets Coordinates
            building[name]['Longitude'] = (nomi.longitude)
            building[name]['Latitude'] = (nomi.latitude)
            building[name]['Emissions'] = emissions
            building[name]['Energy'] = energy
            building[name]['Efficiency'] = emissions/(energy + 1)

for place, value in building.items():
    a = building[place]['Longitude']
    b = building[place]['Latitude']
    c = building[place]['Emissions']
    d = building[place]['Energy']

    #e = float(c)/float(d)
    #print(place)
    #print(f'\t Coordinates:{a}, {b} - Emissions:{c} - Energy:{d} = {e}')

#Makes a table displaying information
df = pd.DataFrame(building).T

df

,Emissions,Latitude,Longitude,Energy,Efficiency
Stephenson Building,0.0,43.6641,-79.3889,2.181065e+06,0.000000
2 Civic Centre Court,115197.0,43.6437,-79.5767,1.259537e+06,0.091460
51 Lisgar,12565.0,43.6480,-79.4177,7.094746e+04,0.177100
Archives and Records Centre,108376.0,43.6736,-79.4035,4.801582e+05,0.225708
Atlantic Ave Storage Bldg,55473.0,43.6383,-79.4301,1.056177e+05,0.525219
...,...,...,...,...,...
West Toronto Pumping Station,178196.0,43.6748,-79.4839,4.454903e+06,0.040000
F.J. Horgan Filtration Plant,2062673.0,43.7678,-79.1866,3.965985e+07,0.052009
Island Filtration Plant,367898.0,43.6230,-79.3936,7.770374e+06,0.047346
R.C. Harris Filtration Plant,1918036.0,43.6784,-79.2941,4.589065e+07,0.041796


In [5]:
#This cell organizes data for green roofs
import csv, pgeocode
from collections import defaultdict

roof = {}

with open ('data/Green Roofs.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        complete = line["COMPLETED_DATE"]
        postal = line["POSTAL"]
        if not complete == "" and not postal == "   ":
            street = line["STREET_NAME"]
            area = float(line['GREEN_ROOF_AREA'].replace(",", ""))
            
            roof.setdefault(street, {
                'Area': 0,
                'Latitude': 0,
                'Longitude': 0
            })
            
            nomi = pgeocode.Nominatim('ca')
            nomi = nomi.query_postal_code(postal)
            roof[street]['Longitude'] = (nomi.longitude)
            roof[street]['Latitude'] = (nomi.latitude)
            roof[street]['Area'] = area

roof = dict(sorted(roof.items(), key = lambda kv: kv[1]['Area'], reverse=True))
max_area = roof[next(iter(roof))]['Area'] 


In [6]:
#This cell maps greenhouse locations vs energy use
from ipyleaflet import Map, Marker, basemaps, Popup, Circle
from ipywidgets import HTML

m = Map(center = (43.6487, -79.38544), zoom = 10, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)
#Sorts dictionary by highest to lowest emissions so on the map the big circle appears below
building = dict(sorted(building.items(), key = lambda kv: kv[1]['Efficiency'], reverse=False))

# Adds circle markers for green roofs
for street, value in roof.items():
    area = roof[street]['Area']
    if area > 0:
        circle = Circle()
        circle.location = (roof[street]['Latitude'], roof[street]['Longitude'])
        circle.radius = int(clamp(1000 * area/2000, 100, 1000))
        #Calculates the color based on greenhouse gas emissions, higher means more red

        col = "Green"
        circle.color = col
        circle.fill_color = col
    
        #Creates a popup for each circle
        popup2 = Popup(
            location=(roof[street]['Latitude'], roof[street]['Longitude']),
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )
    
        #Creates message, which shows division and percentage
        message = HTML()
        message.value = (f'\t{street}, {area} m^2')
    
        m.add_layer(circle)
        circle.popup = message
#Adds circle markers for energy consumtion
for place, value in building.items():
    circle = Circle()
    circle.location = (building[place]['Latitude'], building[place]['Longitude'])
    nrg = building[place]['Energy']
    ghg = building[place]['Emissions']
    circle.radius = int(clamp(math.pow(nrg, 1.0/3), 50, 1750))
    #Calculates the color based on ghg/energy, higher means more red
    green = str(hex(int(clamp(1 - ghg/(nrg + 1), 0, 0.5) * 255 * 2)))
    if int(green, 16) < 16:
        green = "0" + green
    green = green.replace("0x", "")
    
    red = str(hex(int(clamp(ghg/(nrg + 1), 0, 0.5) * 255 * 2)))
    if int(red,16) < 16:
        red = "0" + red
    red = red.replace("0x", "")
    
    col = "#" + red + green + "00"
    circle.color = col
    circle.fill_color = col
    
    #Creates a popup for each circle
    popup2 = Popup(
        location=(building[place]['Latitude'], building[place]['Longitude']),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    
    #Creates message, which shows division and percentage
    message2 = HTML()
    x = ghg/(nrg + 1)
    message2.value = (f'\t{place}, {nrg} kWh, {x}')
    
    m.add_layer(circle)
    circle.popup = message2
m

Map(center=[43.6487, -79.38544], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [7]:
#This cell maps renewables vs GHG emissions
from ipyleaflet import Map, Marker, basemaps, Popup, Circle
from ipywidgets import HTML

highEmission = 3000000
#Sorts dictionary by highest to lowest emissions so on the map the big circle appears below
building = dict(sorted(building.items(), key = lambda kv: kv[1]['Emissions'], reverse=True))

#Creates map
m = Map(center = (43.6487, -79.38544), zoom = 10, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)
#m = Map(center = (43.6656, -79.383), zoom = 10, min_zoom = 1, max_zoom = 20)

#Adds markers
#https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html
for i, value in renewable.items():
    #Stores coordinates in integers
    a = renewable[i]['Latitude']
    b = renewable[i]['Longitude']
    
    #Adds all the markers
    center = (a, b)
    marker = Marker(location=center, draggable=False)
    m.add_layer(marker);
    
    #Creates message
    message = HTML()
    message.value = i
    
    #Creates popup
    popup = Popup(
        location=center,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    #Adds popup to layer and associates it with the message and marker
    marker.popup = message
    
# Adds circle markers
for place, value in building.items():
    circle = Circle()
    circle.location = (building[place]['Latitude'], building[place]['Longitude'])
    ghg = building[place]['Emissions']
    circle.radius = int(clamp(math.sqrt(ghg), 50, 1750))
    #Calculates the color based on greenhouse gas emissions, higher means more red
    green = str(hex(int(clamp(highEmission - ghg, 0, highEmission/2) * 2 * 255/highEmission)))
    if int(green, 16) < 16:
        green = "0" + green
    green = green.replace("0x", "")
    
    red = str(hex(int(clamp(ghg, 0, highEmission/2) * 2 * 255/highEmission)))
    if int(red,16) < 16:
        red = "0" + red
    red = red.replace("0x", "")
    
    col = "#" + red + green + "00"
    circle.color = col
    circle.fill_color = col
    
    #Creates a popup for each circle
    popup2 = Popup(
        location=(building[place]['Latitude'], building[place]['Longitude']),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    
    #Creates message, which shows division and percentage
    message2 = HTML()
    name = place
    message2.value = (f'\t{name}, {ghg} kg')
    
    m.add_layer(circle)
    circle.popup = message2

#Draws Map
m

Map(center=[43.6487, -79.38544], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

<h1>Renewables Analysis<h1>
    <h2> sdFSDF <h2>

In [8]:
#This cell stores information on the age and income of people in the survey and whether they deny climate change
import csv, pgeocode
import numpy as np, pandas as pd
from collections import defaultdict

person = {}
how_old = list()
educated = list()
graph = {}

with open ('data/Climate Perceptions.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        record = line["\ufeffrecord"]
        age = line["HIDAGE1"]
        postal = line["QS1"]
        concern = line["Q1r2"]
        knowledge = line["Q2"]
    
        if concern == "Extremely concerned":
            concern = 5
        elif concern == "Very concerned":
            concern = 4
        elif concern == "Don't know":
            concern = 3
        elif concern == "Not very concerned":
            concern = 2
        else:
            concern = 1
        
        if knowledge == "Extremely informed":
            knowledge = 4
        elif knowledge == "Very informed":
            knowledge = 3
        elif knowledge == "Not very informed":
            knowledge = 2
        else:
            knowledge = 1
        
        person.setdefault(record, {
            'Age': 0,
            'Concern': 0,
            'Knowledge': 0,
            'Latitude': 0,
            'Longitude': 0
        })
        
        nomi = pgeocode.Nominatim('ca')
        nomi = nomi.query_postal_code(postal)
        
        person[record]['Age'] = age
        person[record]['Concern'] = concern
        person[record]['Knowledge']  = knowledge
        person[record]['Latitude'] = nomi.latitude
        person[record]['Longitude'] = nomi.longitude
        how_old.append(age)
        educated.append(knowledge)
        
#graph = pd.DataFrame({'Age': how_old, 'Knowledge': educated})
#graph.plot.scatter(x = 'Age', y = 'Knowledge')